In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import cv2
from keras.applications.xception import Xception,preprocess_input
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D

Using TensorFlow backend.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img

In [0]:
from keras.applications import VGG16

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 5s 0us/step


In [0]:
vgg_conv.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
train_dir = 'drive/My Drive/dataset1/train'
validation_dir = 'drive/My Drive/dataset1/test'

nTrain = 5071
nVal = 1269

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20


train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,6))

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

Found 5071 images belonging to 6 classes.


In [0]:
validation_features = np.zeros(shape=(nVal, 7, 7, 512))
validation_labels = np.zeros(shape=(nVal,6))

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, 7 * 7 * 512))

Found 1269 images belonging to 6 classes.


In [0]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features,
                    train_labels,
                    epochs=20,
                    batch_size=20,
                    validation_data=(validation_features,validation_labels))


Train on 5071 samples, validate on 1269 samples
Epoch 1/20
5071/5071 [==============================] - 6s 1ms/step - loss: 0.6110 - acc: 0.8318 - val_loss: 0.1586 - val_acc: 0.9409
Epoch 2/20
5071/5071 [==============================] - 5s 985us/step - loss: 0.1716 - acc: 0.9397 - val_loss: 0.1149 - val_acc: 0.9622
Epoch 3/20
5071/5071 [==============================] - 5s 979us/step - loss: 0.1039 - acc: 0.9645 - val_loss: 0.1975 - val_acc: 0.9425
Epoch 4/20
5071/5071 [==============================] - 5s 979us/step - loss: 0.0809 - acc: 0.9708 - val_loss: 0.0810 - val_acc: 0.9732
Epoch 5/20
5071/5071 [==============================] - 5s 986us/step - loss: 0.0598 - acc: 0.9781 - val_loss: 0.0791 - val_acc: 0.9764
Epoch 6/20
5071/5071 [==============================] - 5s 980us/step - loss: 0.0526 - acc: 0.9828 - val_loss: 0.0608 - val_acc: 0.9842
Epoch 7/20
5071/5071 [==============================] - 5s 983us/step - loss: 0.0389 - acc: 0.9854 - val_loss: 0.0500 - val_acc: 0.9866
Ep